In [1]:
from copy import deepcopy
import glob
import json
import os
import pickle
import shutil

from PIL import Image

In [22]:
anno_path = 'DATA/test_hico_ann.json'  # hico-det annotation file
output_path = 'generated_output/'  # directory of generated images,
new_dataset_path = '../FGAHOI/data/id_generated_output/'  # target FGAHOI-formatted dataset path
res_multi = pickle.load(open('DATA/hico_det_test.pkl', 'rb'))
annos = json.load(open(anno_path))
print(len(res_multi))
assert len(glob.glob(output_path+'test2015/*.jpg')) == len(res_multi), f"{len(glob.glob(output_path+'test2015/*.jpg'))} != {len(res_multi)}"

9546


In [ ]:
img_id_to_annos_idx_map = {x["img_id"]: i for i, x in enumerate(annos)}
anno_list = json.load(open('../FGAHOI/data/hico_20160224_det/annotations/anno_list.json', 'r'))
anno_list_to_idx_map = {x["global_id"]: i for i, x in enumerate(anno_list)}

In [26]:
def listdir_nohidden(path, search="*"):
    return glob.glob(os.path.join(path, search))
def convert_bbox(anno, new_width, new_height):
    annotations = anno['annotations']
    for an in annotations:
        an['bbox'] = [new_width*an['bbox'][0]/anno['width'],
                      new_height*an['bbox'][1]/anno['height'],
                      new_width*an['bbox'][2]/anno['width'],
                      new_height*an['bbox'][3]//anno['height']]
        an['bbox'] = [int(a) for a in an['bbox']]
    anno['width'] = new_width
    anno['height'] = new_height
def generate_new_anno(img_id, idx, annos):
    assert idx in [0,1]
    new_anno = deepcopy(annos[img_id_to_annos_idx_map[img_id]])
    convert_bbox(new_anno, new_width=512, new_height=512)
    return new_anno

In [27]:
new_annos_1 = []
dst = new_dataset_path+'images/test2015'
os.makedirs(os.path.dirname(dst), exist_ok=True)
if not os.path.exists(dst):
    os.symlink(os.path.abspath(output_path+'test2015/'), dst)
new_anno_list_1 = deepcopy(anno_list)

In [29]:
assert len(glob.glob(output_path+'test2015/*.jpg')) == len(res_multi), f"{len(glob.glob(output_path+'test2015/*.jpg'))} != {len(res_multi)}"
for anno in res_multi:
    img_id = anno['img_id']
    generated_imgs = listdir_nohidden(output_path+'test2015/', f'{anno["file_name"]}')
    if not len(generated_imgs) == 1:
        print(f"{img_id} {generated_imgs} {anno}")
        continue
    new_annos_1.append(generate_new_anno(img_id, 0, annos))

    t = new_anno_list_1[anno_list_to_idx_map[anno['file_name'].split('.')[0]]]

    ori_width = t['image_size'][1]
    ori_height = t['image_size'][0]
    new_height, new_width = 512, 512
    for hoi in t['hois']:
        for i, bbox in enumerate(hoi['human_bboxes']):
            bbox = [new_width*bbox[0]/ori_width,
                    new_height*bbox[1]/ori_height,
                    new_width*bbox[2]/ori_width,
                    new_height*bbox[3]/ori_height]
            bbox = [int(a) for a in bbox]
            hoi['human_bboxes'][i] = bbox
        for i, bbox in enumerate(hoi['object_bboxes']):
            bbox = [new_width*bbox[0]/ori_width,
                    new_height*bbox[1]/ori_height,
                    new_width*bbox[2]/ori_width,
                    new_height*bbox[3]/ori_height]
            bbox = [int(a) for a in bbox]
            hoi['object_bboxes'][i] = bbox
    t['image_size'] = [new_height, new_width, 3]

In [30]:
os.makedirs(os.path.dirname(new_dataset_path+'/annotations/'), exist_ok=True)

In [31]:
anno_list1_path = os.path.join(new_dataset_path, "annotations/anno_list.json")
json.dump(new_anno_list_1, open(anno_list1_path, 'w'))

In [33]:
anno1_path = os.path.join(new_dataset_path, "annotations/test_hico.json")
os.makedirs(os.path.dirname(anno1_path), exist_ok=True)
json.dump(new_annos_1, open(anno1_path, 'w'))

In [34]:
# copy other annotations

In [35]:
to_copy_over = listdir_nohidden('../FGAHOI/data/hico_20160224_det/annotations/',
                                search="[!tr|anno]*")

In [36]:
to_copy_over

['FGAHOI/data/hico_20160224_det/annotations/hoi_list_new.json',
 'FGAHOI/data/hico_20160224_det/annotations/corre_hico.npy',
 'FGAHOI/data/hico_20160224_det/annotations/hoi_id_to_num.json',
 'FGAHOI/data/hico_20160224_det/annotations/corre_hico_1.npy',
 'FGAHOI/data/hico_20160224_det/annotations/file_name_to_obj_cat.json']

In [37]:
for file in to_copy_over:
    shutil.copy2(file, new_dataset_path+'/annotations/')

In [ ]:
file = '../FGAHOI/data/hico_20160224_det/annotations/trainval_hico.json'
shutil.copy2(file, new_dataset_path+'/annotations/')

In [39]:
print("Done")

Done
